In [2]:
import tensorflow as tf
import numpy as np
import scipy.special

In [3]:
N = 8
K = 4

In [4]:
def full_adder(a,b,c):
    s = (a ^ b) ^ c
    c = (a & b) | (c & (a ^ b))
    return s,c

def add_bool(a,b):
    k = len(a)
    s = np.zeros(k,dtype=bool)
    c = False
    for i in reversed(range(0,k)):
        s[i], c = full_adder(a[i],b[i],c)    
    
    return s

def inc_bin(a):
    k = len(a)
    increment = np.hstack((np.zeros(k-1,dtype=int), np.ones(1,dtype=int)))
    a = add_bool(a,increment)
    return a

def approx(C):
    W = inv_fc(1-(1-pi_fc(C))**2)  
    return W

def pi_fc(x):
    a = -0.4527
    b = 0.0218
    r = 0.86
    
    if x < 10:
        P = np.exp(a*(x**r)+b)
    else:
        P = np.sqrt((np.pi)/x)*np.exp(-x/4)*(1-10/(7*x))
        
    return P

def inv_fc(x):
    a = -0.4527
    b = 0.0218
    r = 0.86
    
    I = ((np.log(x)-b)/a)**(1/r)
    
    return I
    

def polarization(N, K, sigma_train):
    
    W = np.ones(N, dtype=float)
    W_temp = np.zeros(N, dtype=float)
    
    W[0] = (2/(sigma_train**2))
    
    for i in range(1, int(np.log2(N))+1):
        W_temp[:] = W[:]
        t = 2**(i-1)
        
        for j in range(1,(int(t))+1):
            C = W_temp[j-1]
            W[2*j-1-1] = approx(C)         #polarization by using density evolution
            W[2*j-1] = 2*C
                  
    qfunc = lambda x: 0.5-0.5*scipy.special.erf(x/np.sqrt(2))
    W = qfunc(np.sqrt(W/2))
    idx = sorted(range(W.size), key=lambda k: -W[k]) # descend order 
    A = idx[-K:]

    
    return A
    
def encoding(u, N):

    n = 1
    x = np.copy(u)
    step = np.log2(N)
    for s in range(0,step.astype(int)):
        i = 0
        while i < N:
            for j in range(0,n):
                idx = i+j
                x[idx] = x[idx] ^ x[idx+n]
            i=i+2*n
        n=2*n
        
    for s in range(0,N):
        if np.mod(x[s],2) == 0:
            x[s] = 0

    return x

In [5]:
snr_train = 2
sigma_train = np.sqrt(1/(2*10**(snr_train/10)))
print(sigma_train)

0.561674881261


In [6]:
b = np.zeros((2**K,K),dtype=int)
for i in range(1,2**K):
    b[i]= inc_bin(b[i-1])

In [7]:
A = polarization(N, K, sigma_train)
u = np.zeros((2**K, N), dtype=int)
u[:,A] = b

In [8]:
A

[3, 5, 6, 7]

In [9]:
x = np.zeros((2**K, N), dtype=int)

for i in range(0, 2**K):
    x[i,:] = encoding(u[i,:], N)
    
x = (-1)**x # BPSK modulation

In [10]:
b[:,:]

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

In [11]:
u[:,:]

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1, 1]])

In [12]:
x[:,:]

array([[ 1,  1,  1,  1,  1,  1,  1,  1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  1, -1,  1, -1,  1, -1,  1],
       [ 1, -1,  1, -1,  1, -1,  1, -1],
       [-1, -1,  1,  1, -1, -1,  1,  1],
       [ 1,  1, -1, -1,  1,  1, -1, -1],
       [ 1, -1, -1,  1,  1, -1, -1,  1],
       [-1,  1,  1, -1, -1,  1,  1, -1],
       [-1, -1, -1, -1,  1,  1,  1,  1],
       [ 1,  1,  1,  1, -1, -1, -1, -1],
       [ 1, -1,  1, -1, -1,  1, -1,  1],
       [-1,  1, -1,  1,  1, -1,  1, -1],
       [ 1,  1, -1, -1, -1, -1,  1,  1],
       [-1, -1,  1,  1,  1,  1, -1, -1],
       [-1,  1,  1, -1,  1, -1, -1,  1],
       [ 1, -1, -1,  1, -1,  1,  1, -1]])

In [20]:
Layer1 = N
Layer2 = N
Layer3 = N
Layer4 = N


X = tf.placeholder(tf.float32,[None, N])
Y = tf.placeholder(tf.float32,[None, N])

W1 = tf.Variable(tf.random_uniform([N, Layer1], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform([Layer1, Layer2], -1.0, 1.0))
W3 = tf.Variable(tf.random_uniform([Layer2, Layer3], -1.0, 1.0))
W4 = tf.Variable(tf.random_uniform([Layer3, N], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([Layer1]), name = "Bias1")
b2 = tf.Variable(tf.zeros([Layer2]), name = "Bias2")
b3 = tf.Variable(tf.zeros([Layer3]), name = "Bias3")
b4 = tf.Variable(tf.zeros([Layer4]), name = "Bias4")

L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
hypo = tf.nn.sigmoid(tf.matmul(L3, W4) + b4)

#cost = -tf.reduce_mean(tf.maximum(Y,0)*tf.log(hypo)+tf.maximum(-Y,0)*tf.log(-hypo)+((1-tf.abs(Y))*tf.log(1-hypo)))
#cost = -tf.reduce_mean(tf.maximum(Y,0)*tf.log(hypo)+tf.maximum(-Y,0)*tf.log(-hypo)+((1-tf.abs(Y))*tf.log(1-hypo)))
#cost = -tf.reduce_mean(Y*tf.log(hypo)+(1-Y)*tf.log(1-hypo))
cost = tf.reduce_mean(tf.square(Y-hypo))

a = 0.01
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.global_variables_initializer()

In [21]:
sess = tf.Session()
sess.run(init)

sigma = sigma_train

for t in range(5):
    
    print("epoch:", 2**(2*t+10))
    
    for step in range(2**(2*t+10)):
        sess.run(train, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u})

        if step % (np.minimum(2**(2*t+10)-1, 10000)) == 0:
            print("--",step,"->" ,sess.run(cost, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u}))
            

    #correct_prediction = tf.equal(tf.round(2*hypo-1), Y)
    correct_prediction = tf.equal(tf.round(hypo), Y)

    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    
    print("cost:", sess.run(cost, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u}))
    print("Accuracy:", sess.run(accuracy, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u}))
    print("-----------------------------------------")



epoch: 1024
-- 0 -> 0.444688
-- 1023 -> 0.271373
cost: 0.252981
Accuracy: 0.65625
-----------------------------------------
epoch: 4096
-- 0 -> 0.273244
-- 4095 -> 0.133913
cost: 0.156814
Accuracy: 0.796875
-----------------------------------------
epoch: 16384
-- 0 -> 0.128879
-- 10000 -> 0.0813735
cost: 0.0733791
Accuracy: 0.890625
-----------------------------------------
epoch: 65536
-- 0 -> 0.0698437
-- 10000 -> 0.0775487
-- 20000 -> 0.0565692
-- 30000 -> 0.0621673
-- 40000 -> 0.052276
-- 50000 -> 0.0418225
-- 60000 -> 0.0301563
cost: 0.0236435
Accuracy: 0.953125
-----------------------------------------
epoch: 262144
-- 0 -> 0.0349099
-- 10000 -> 0.0416556
-- 20000 -> 0.0219327
-- 30000 -> 0.0307177
-- 40000 -> 0.015426
-- 50000 -> 0.0193253
-- 60000 -> 0.0255194
-- 70000 -> 0.0279481
-- 80000 -> 0.0129098
-- 90000 -> 0.0154889
-- 100000 -> 0.0304981
-- 110000 -> 0.0149551
-- 120000 -> 0.0118097
-- 130000 -> 0.0152324
-- 140000 -> 0.00816998
-- 150000 -> 0.0142979
-- 160000 -> 0.